![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [2]:
# dataset
PATH = "/content/arabic words excel (2).xlsx"

Raniya is trying

In [3]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.2 MB/s eta 0:00:00


In [4]:
def preprocessing(sentence):
    # Add space after colon
    sentence = sentence.replace(':', ' : ')
    # Remove extra space between words
    sentence = ' '.join(sentence.split())
    # Remove Diacritics
    sentence = araby.strip_diacritics(sentence)
    # Replace period wiht new line
    sentence = sentence.replace('.', '\n')
    # Remove extra new lines after the last word
    sentence = sentence.rstrip()
    return sentence




In [5]:
import pandas as pd
import numpy as np
import pyarabic.araby as araby

np.random.seed(42)

df = pd.read_excel(PATH)

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# Data preprocessing
df['الإعراب'] = df['الإعراب'].apply(lambda x: preprocessing(x))
df['الجملة'] = df['الجملة'].apply(lambda x: preprocessing(x))

df.head()

,الجملة,الإعراب,المصدر
0,قال الطالب : أنا ناجح,قال : فعل ماضي\nالطالب : فاعل مرفوع وعلامة رفع...,NaN
1,المرضى نائمون,المرضى : مبتدأ مرفوع وعلامة رفعه الضمة المقدرة...,NaN
2,الطائر يطير في السماء,الطائر : مبتدأ مرفوع وعلامة رفعه الضمة الظاهرة...,NaN
3,الذي يفوز فله جائزة,الذي : اسم موصول متضمن معنى الشرط،مبني على الس...,NaN
4,إن العلم طريقه طويلة,إن : حرف توكيد ونصب مبني على الفتحة لا محل له ...,NaN


# Generate wrong samples ( word level )


In [6]:
import random

words = [sentence.split('\n') for sentence in list(df['الإعراب'])]
sen_words = {sen:words for sen,words in zip(list(df['الجملة']),words)}
# words

In [7]:
random.seed(42)


def generate_random_irab(word):


    roles = [
    "اسم موصول", "مفعول فيه", "ظرف زمان", "مبتدأ", "خبر", "فاعل", "مفعول به",
    "حال", "تمييز", "مضاف إليه", "نعت", "عطف بيان", "توكيد", "بدل", "مفعول مطلق",
    "مفعول لأجله", "مفعول معه", "مستثنى", "منادى", "اسم إن", "خبر إن", "اسم كان",
    "خبر كان", "نائب فاعل", "حرف جر", "حرف توكيد ونصب", "حرف عطف","اسم مجرور"
    ]

    cases = ["مرفوع", "منصوب", "مجرور", "مجزوم", "مبني"]

    case_markers = {
        "مرفوع": ["الضمة", "الواو", "الألف", "ثبوت النون"],
        "منصوب": ["الفتحة", "الألف", "الياء", "الكسرة", "حذف النون"],
        "مجرور": ["الكسرة", "الياء", "الفتحة"],
        "مجزوم": ["السكون", "حذف حرف العلة", "حذف النون"]
    }

    results = []
    role = random.choice(roles)
    case = random.choice(cases)

    if case == "مبني":
        case_description = f"مبني على {random.choice(['الضم', 'الفتح', 'الكسر', 'السكون'])}"
    else:
        # Based on the case, choose a random case marker that could come with this case
        marker = random.choice(case_markers.get(case, ["علامة غير محددة"]))

        # I think this is clear no need for comments
        case_marker = {
            "مرفوع": "رفعه",
            "منصوب": "نصبه",
            "مجرور": "جره",
            "مجزوم": "جزمه"
        }.get(case, "إعرابه")
        case_description = f"{case} وعلامة {case_marker} {marker}"

    irab = f"{word} : {role} {case_description}"

    # Remove repeated words (just to make sure that the case اسم مجرور مجرور will be اسم مجرور)
    irab = irab.split()
    irab = list(dict.fromkeys(irab))
    irab = ' '.join(irab)
    return irab

wrong_sen_words = {}
for sen,words in sen_words.items():
    wrong_words = []
    for word in words:
        word = word.split(' : ')[0]
        wrong_words.append(generate_random_irab(word))
    wrong_sen_words[sen] = wrong_words

# df['اعراب خاطئ'] = wrong_examples
# df['اعراب صحيح'] = words
# df.head()
sen_words[sen]

['دائما : مفعول فيه ظرف زمان، متعلق بالفعل أحب',
 'أحب : فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة، والفاعل ضمير مستتر فيه وجوبا تقديره "أنا"',
 'والدي : مفعول به منصوب وعلامة نصبه الفتحة المقدرة على ما قبل الياء منع من ظهورها اشتغال المحل بالحركة المناسبة للياء،وهومضاف،الياء : ضميرمتصل مبني على السكون في محل جر مضاف اليه']

# API key

In [8]:
# with open('examples.txt','w') as f:
#         f.write(examples[0])

In [9]:
import os
import getpass
key = ''
def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass(f"{key}")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [10]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [78]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [65]:
project_id = ''
space_id = None


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [66]:
from IPython.display import clear_output
!pip install ibm_watsonx_ai
clear_output()

In [ ]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [87]:
def prompt_name(name):
  few_shots = """
<s> [INST]
        الجمله: ما أهملت واجبى قط
        الاعراب: ما : اسم إن منصوب وعلامة نصبه الألف
 [/INST]
الإعراب الصحيح: ما: اسم نكرة مبني على السكون في محل رفع مبتدأ
خاطئ
</s><s> [INST]
الجمله: الطائر يطير في السماء
الاعراب الصحيح: يطير : فعل مضارع مرفوع وعلامة رفعه الضمة الظاهرة على آخره،والفاعل ضمير مستتر تقديره "هو" يعود على الطائر
[/INST]
صحيح
</s><s> [INST]
الجمله: الذي يفوز فله جائزة
الاعراب: يفوز : فعل مضارع مرفوع،وعلامة رفعه الضمة الظاهرة على آخره
[/INST]
صحيح
</s><s> [INST]
الجمله: كل من تلقاه يشكو دهره
الاعراب: يشكو : فعل مضارع مرفوع وعلامة رفعه الضمة المقدرة على الواو للثقل،والفاعل ضمير مستتر تقديره "هو"
 [/INST]
صحيح
</s><s> [INST]
الجمله: خرج القطار من المحطة
الاعراب: المحطة : مفعول لأجله مرفوع وعلامة رفعه الضمة
[/INST]
الاعراب الصحيح: المحطة: مفعول به منصوب وعلامة نصبه الفتحة
خاطئ
</s><s> [INST]
الجمله: دائما أحب والدى
الاعراب: والدي : توكيد منصوب وعلامة نصبه الكسرة
[/INST]
الإعراب الصحيح: والدي: مفعول به منصوب وعلامة نصبه الفتحة
خاطئ
</s><s> [INST]
الجمله: دائما أحب والدى
الاعراب: والدي : توكيد منصوب وعلامة نصبه الكسرة
[/INST]
الإعراب الصحيح: أحب : خبر كان منصوب وعلامة نصبه حذف النون
خاطئ
</s><s> [INST]
الجمله: دائما أحب والدى
الإعراب الصحيح: دائما : مفعول مطلق مجرور وعلامة جره الكسرة
[/INST]
الإعراب الصحيح: دائما: ظرف زمان منصوب وعلامة نصبه الفتحة
خاطئ
</s><s> [INST]
الجمله: الأولاد يلعبون في الحديقة
الاعراب: الحديقة : حرف جر مبني على السكون
[/INST]
الإعراب الصحيح: الحديقة : اسم مجرور وعلامة جره الكسرة الظاهرة على آخره
خاطئ.
<s></s> [INST]
الجمله: الأولاد يلعبون في الحديقة
الاعراب: في : عطف بيان مجرور وعلامة جره الفتحة
[/INST]
الإعراب الصحيح: حرف جر
خاطئ
</s>"""

  zero_shots = """
  حدد اذا كان الاعراب المعطى للكلمة صحيحه ام خاطئة, فقط اجب بصحيح او خاطئ
  اذا كان الاعراب المعطى خاطئ, اجب ب خاطئ مع تصحيح الاعراب
  """
  cot = switch_case("COT_few_shots")
  dic = {'cot':cot,"zero_shots":zero_shots,"few_shots":few_shots,'zero_cot':zero_shots}
  return dic[name]


In [88]:
all_prompts = ['zero_shots','few_shots','cot',"zero_cot"]





## Evaluation



In [91]:
questions = []

for sen,words in sen_words.items():
    for word in words:
        question = f"""
        الجمله: {sen}
        الاعراب: {word}"""
        questions.append(question)

for sen,words in wrong_sen_words.items():
    for word in words:
        question = f"""
        الجمله: {sen}
        الاعراب: {word}"""
        questions.append(question)

gt = [1]*(int(len(questions)/2)) +  [0]*(int(len(questions)/2))

In [36]:
"""\n\n"""

'\n\n'

In [ ]:
!pip install wandb


In [38]:

import wandb

# Replace with your actual API key
wandb_api_key = ""

wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [75]:
def accuracy(ground_truth, predections):
    return float((np.array(ground_truth)==np.array(predections)).sum()/len(ground_truth))



In [94]:
import time
for name in all_prompts:
  preds = []
  prompt = prompt_name(name)

  wandb.init(
      # set the wandb project where this run will be logged
      project=name,

  )

  all_times = 0
  for i,q in enumerate(questions):
      question = f"""{prompt}<s> [INST]
      {q} [/INST]"""
      if name == 'zero_cot':
        question = f"""<s> [INST] {prompt}
        {q}
        لنقم بتحليل الجملة خطوة بخطوة:
        [/INST]"""
        pass
      start = time.time()
      print(question[-50:])
      generated_response = model.generate_text(prompt=question, guardrails=True)
      end = time.time()
      wandb.log({"response_time": end-start})
      all_times += end-start
      if "خاطئ" in generated_response or "غير صحيح" in generated_response:
          preds.append(0)
      elif "صحيح" in generated_response:
          preds.append(1)
      else:
          preds.append(2)
      if i%50 == 0:
        print(generated_response)
      print(f'{i+1}/{len(questions)}')

  wandb.log({"response_time_AVG": all_times/len(questions)})
  print(f'{name} response_time_AVG: {all_times/len(questions)}')
  wandb.log({'accuracy': {accuracy(gt,preds)}})
  print(f'{name} accuracy: {accuracy(gt,preds)}')

  wandb.finish()


response_time,█▃▅▂▅▁▇▆▆▃
response_time,1.50289


: أنا ناجح
        الاعراب: قال : فعل ماضي [/INST]
 صحيح

الإعراب الصحيح: قال: فعل ماضٍ مبني على الفتح
خاطئ

في الجملة "قال الطالب: أنا ناجح"، الفعل "قال" هو فعل ماضٍ مبني على الفتح. الفاعل في هذه الجملة هو "الطالب" وهو مرفوع وعلامة رفعه الضمة الظاهرة على آخره. أما الجملة "أنا ناجح" فهي جملة اسمية وليست فعلية، لذا لا يوجد إعراب لها في هذا السياق. 
1/744
راب: الطالب : فاعل مرفوع وعلامة رفعه الضمة [/INST]
2/744
 الاعراب: أنا : ضمير مبني في محل رفع مبتدأ [/INST]
3/744
لاعراب: ناجح : خبر مرفوع وعلامة رفعه الضمة [/INST]
4/744
 مرفوع وعلامة رفعه الضمة المقدرة على الألف [/INST]
5/744
مرفوع وعلامة رفعه الواو لأنه جمع مذكر سالم [/INST]
6/744
أ مرفوع وعلامة رفعه الضمة الظاهرة على آخره [/INST]
7/744
اعل ضمير مستتر تقديره "هو" يعود على الطائر [/INST]
8/744
طير في السماء
        الاعراب: في : حرف جر [/INST]
9/744
بـ "في" وعلامة جره الكسرة الظاهرة على آخره [/INST]
10/744
عنى الشرط،مبني على السكون في محل رفع مبتدأ [/INST]
11/744
ع مرفوع،وعلامة رفعه الضمة الظاهرة على آخره [/INST]
12/744
راب: الفاعل : 

Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"8604bef177810cec21ec97a16a0b1dcf","status_code":400}


107/744
ر متصل مبني على السكون في محل نصب مفعول به [/INST]


Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"2febd6178fe909133e9d7d30a11592b3","status_code":400}


...

Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"3efcb474934557cc147192d32a83ec4e","status_code":400}
Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"1ef4b99ba2293226755ef1b86032af42","status_code":400}
Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"9775f2fc5883d14e125c7401b4e8436b","status_code":400}


334/744
ل مرفوع وعلامة رفعه الضمة الظاهرة على آخره [/INST]
335/744
 منصوب وعلامة نصبه الفتحة الظاهرة على آخره [/INST]
336/744
 على السبورة
        الاعراب: على : حرف جر [/INST]
337/744
ـ "على" وعلامة جره الكسرة الظاهرة على آخره [/INST]
338/744
 : ضمير متصل مبني على الضم في محل رفع فاعل [/INST]
339/744
 منصوب وعلامة نصبه الفتحة الظاهرة على آخره [/INST]
340/744
لامة نصبه الفتحة الظاهرة على آخره،وهو مضاف [/INST]
341/744
رة على آخره،والفاعل ضمير مستتر تقديره "هو" [/INST]
342/744
راب: هذا : اسم إشارة مبني في محل رفع مبتدأ [/INST]


Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"f8ea3f8aea7fe26e85d34e8cf71affac","status_code":400}
Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"644bc382cd7eac99a96053b8222d1dfb","status_code":400}
Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty in the request body","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai"}],"trace":"f713325801ab5b371ae6027d17c0c6aa","status_code":400}
Status code: 400, body: {"errors":[{"code":"json_validation_error","message":"Json document validation error: 'input' cannot be missing or empty 

343/744
: خبر المبتدأ مرفوع وعلامة رفعه تنوين الضم [/INST]
344/744
اهرة على آخره،والفاعل ضمير مستتر تقديره هو [/INST]
345/744
 المقاربة مبني على الفتحة الظاهرة على آخره [/INST]
346/744
د مرفوع وعلامة رفعه الضمة الظاهرة على آخره [/INST]
347/744
 ظهورها الثقل،والفاعل ضمير مستتر تقديره هو [/INST]
348/744
ث حرف مبني على السكون لا محل له من الإعراب [/INST]
349/744
ل مرفوع وعلامة رفعة الضمة الظاهرة على آخره [/INST]
350/744
 منصوب وعلامة نصبة الفتحة الظاهرة على آخره [/INST]

الإعراب الصحيح: رماداً: خبر "استحالت" منصوب وعلامة نصبه الفتحة الظاهرة على آخره
خاطئ

في الجملة "استحالت النار رماداً"، الفعل "استحالت" هو الذي يحتاج إلى خبر. الخبر هو الجزء الذي يخبر عن المبتدأ ويوضح حالته أو صفته. في هذه الجملة، الخبر هو "رماداً" وهو منصوب بالفتحة الظاهرة على آخره. 
351/744
: رأيت : رأى : فعل ماضي،التاء : تاء الفاعل [/INST]
352/744
الطلاب : مفعول به منصوب وعلامة نصبه الفتحة [/INST]
353/744
مسة،الواو : ضمير متصل مبني في محل رفع فاعل [/INST]
354/744
 : هم : ضمير متصل مبني في محل جر مضاف إليه [/INST]
355/7

KeyboardInterrupt: 